In [1]:
import os
import pandas as pd

Current best submission
---

**Status: 4. Mai 2024, 12:45**

*Key: SwinV2-S (All modalities) -- multimodal_swin_05-03_C*

*Score: 0.64655 // MPA_ALL_MODALITIES: 0.65097 // MPA_TOP_VIEW: 0.64213*

Model description
-----
- Model name: **X-Multi-Modal Late Fusion**
  - Orthophotos: SwinV2 transformer (small)
  - Street photos: SwinV2 transformer (small)
  - Sentinel-2 data: SwinV2 transformer (small)

- The pretrained transformers provide embeddings for each modality.
- These are combined through an attention module (mode: attention2).
- At inference time, the model works even when the streetview image is not present.

Data description
---

- All samples from HUN are excluded. They are underrepresented in the test set, and proved very hard to predict even with overfitting on a single batch.
- 5-fold cross validation from StratifiedGroupKFold (stratify on class labels, group cities)
- Data augmentation during training includes horizontal flips and colorjitter.

Training procedure
---
- Early stopping is applied on the validation set confusion matrix diagonal mean.
- Train on full node (2 GPUs).
- Cross-entropy loss weighted with class weights from inverse sample count


Test predictions and post-processing
---

Test set predictions are generated for each fold, and majority vote is applied. Where there is a tie, the value closest to the mean prediction is chosen.

In [2]:
best_submission_root = '../submissions/current_best_model/train/'
best_experiments = os.listdir(best_submission_root)
best_experiments

['multimodal_swin_05-03_C']

In [3]:
runs = os.listdir(os.path.join(best_submission_root, best_experiments[0]))

In [4]:
with open(os.path.join(best_submission_root, best_experiments[0], runs[0], 'config_tree.log')) as f:
    config_tree = f.readlines()

In [5]:
config_tree

['CONFIG\n',
 '├── datamodule\n',
 '│   └── _target_: ai4eo_mapyourcity.datamodules.mapyourcity_datamodule.MapYourCi\n',
 '│       batch_size: 16                                                          \n',
 '│       num_workers: 8                                                          \n',
 '│       pin_memory: false                                                       \n',
 '│       dataset_options:                                                        \n',
 '│         data_dir: /work/ka1176/caroline/gitlab/AI4EO-MapYourCity/scripts/../da\n',
 '│         fold: 3                                                               \n',
 '│         fold_dir: /work/ka1176/caroline/gitlab/AI4EO-MapYourCity/scripts/../da\n',
 '│         fold_key: random_stratified_labels_cities_noHUN                       \n',
 '│         model_id:                                                             \n',
 '│           topview: swinv2_small_window16_256.ms_in1k                          \n',
 '│      

## References

https://openaccess.thecvf.com/content/CVPR2022/papers/Ma_Are_Multimodal_Transformers_Robust_to_Missing_Modality_CVPR_2022_paper.pdf